In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604520,32.3,1292120,69.1,1292120,69.1
Vcells,1086848,8.3,8388608,64.0,1632674,12.5


In [2]:
semilla <- 100019
set.seed(semilla)

In [3]:
#Feature Engineering
#creo nuevas variables dentro del mismo mes
#Condimentar a gusto con nuevas variables

require("data.table")

#Establezco el Working Directory
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory

kcarpeta_datasets    <- "./datasets/"   #KAGGLE
#kcarpeta_datasets    <- "./buckets/b1/datasets/"                          #VM o Ubuntu

#Archivo con datos etiquetados para entrenamiento
karchivo_entrada      <-  paste0(kcarpeta_datasets, "competencia1_2022.csv")
karchivo_salida      <-  paste0(kcarpeta_datasets, "competencia1_2022_FE_DC2.csv")



#lectura rapida del dataset  usando fread  de la libreria  data.table
dataset_orig  <- fread(karchivo_entrada)



Loading required package: data.table



In [4]:
EnriquecerDataset <- function( dataset )
{
  columnas_originales <-  copy(colnames( dataset ))

  #INICIO de la seccion donde se deben hacer cambios con variables nuevas
  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , mv_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , mv_status02       := Master_status +  Visa_status ]
  dataset[ , mv_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , mv_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , mv_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  dataset[ , mv_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]

  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]


    
  #combino MasterCard y Visa
  dataset[ , mv_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , mv_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]

# PROPIAS
    dataset[ , prop_vida_en_banco          := cliente_antiguedad/(12*cliente_edad)]
    
    dataset[ , mas_una_caja_ahorro         := ifelse(ccaja_ahorro > 1,1,0)]
    
    #Prestamos
    dataset[ , cprestamos                 := rowSums( cbind( cprestamos_personales, cprestamos_prendarios, cprestamos_hipotecarios) , na.rm=TRUE ) ]
    dataset[ , mprestamos                 := rowSums( cbind( mprestamos_personales, mprestamos_prendarios, mprestamos_hipotecarios) , na.rm=TRUE ) ]
    dataset[ , tiene_prestamos             := ifelse(cprestamos > 0,1,0)]
    
    #Plazos Fijos
    dataset[ , mplazo_fijo                := rowSums( cbind( mplazo_fijo_pesos, mplazo_fijo_dolares) , na.rm=TRUE ) ]
    dataset[ , tiene_plazos_fijos             := ifelse(cplazo_fijo > 0,1,0)]

    #Inversiones
    dataset[ , minversion1                := rowSums( cbind( minversion1_pesos, minversion1_dolares) , na.rm=TRUE ) ]
    dataset[ , cinversion12                := rowSums( cbind( cinversion1, cinversion2) , na.rm=TRUE ) ]
    dataset[ , minversion12                := rowSums( cbind( minversion1, minversion2) , na.rm=TRUE ) ]
    dataset[ , cinversion_total                := rowSums( cbind( cinversion12, cplazo_fijo) , na.rm=TRUE ) ]
    dataset[ , minversion_total                := rowSums( cbind( minversion12, mplazo_fijo) , na.rm=TRUE ) ]
    dataset[ , tiene_inversiones_total             := ifelse(cinversion_total > 0,1,0)]
    dataset[ , tipo_inversion1_sobre_2             := ifelse(minversion2 == 0,0,minversion1/minversion2)]
    dataset[ , tipo_inversion_pf_sobre_total             := ifelse(minversion_total == 0,0,mplazo_fijo/minversion_total)]

    #Seguros
    dataset[ , cseguro                := rowSums( cbind( cseguro_vida, cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales) , na.rm=TRUE ) ]
    dataset[ , tiene_seguros             := ifelse(cseguro > 0,1,0)]

    #Debitos automaticos
    dataset[ , cdebitos_automaticos                := rowSums( cbind( ccuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos ) , na.rm=TRUE ) ]
    dataset[ , mdebitos_automaticos                := rowSums( cbind( mcuenta_debitos_automaticos,mttarjeta_visa_debitos_automaticos,mttarjeta_master_debitos_automaticos ) , na.rm=TRUE ) ]
    dataset[ , tiene_debitos_automaticos              := ifelse(cdebitos_automaticos > 0,1,0)]

    #Servicios
    dataset[ , cpago_servicios_total                := rowSums( cbind( cpagodeservicios,cpagomiscuentas ) , na.rm=TRUE ) ]
    dataset[ , mpago_servicios_total                := rowSums( cbind( mpagodeservicios,mpagomiscuentas ) , na.rm=TRUE ) ]
    dataset[ , paga_servicios             := ifelse(cpago_servicios_total > 0,1,0)]

    #Descuentos
    dataset[ , cdescuentos                := rowSums( cbind( ccajeros_propios_descuentos,ctarjeta_visa_descuentos,ctarjeta_master_descuentos ) , na.rm=TRUE ) ]
    dataset[ , mdescuentos                := rowSums( cbind( mcajeros_propios_descuentos,mtarjeta_visa_descuentos,mtarjeta_master_descuentos ) , na.rm=TRUE ) ]
    dataset[ , tiene_descuentos             := ifelse(cdescuentos > 0,1,0)]

    #Comisiones
    dataset[ , ccomisiones                := rowSums( cbind( ccomisiones_mantenimiento,ccomisiones_otras ) , na.rm=TRUE ) ]
    dataset[ , mcomisiones                := rowSums( cbind( mcomisiones_mantenimiento,mcomisiones_otras ) , na.rm=TRUE ) ]
    dataset[ , paga_comisiones             := ifelse(ccomisiones > 0,1,0)]

    #Moneda extranjera
    dataset[ , opera_moneda_extranjera             := ifelse(cforex > 0,1,0)]

    #Receptor frecuente
    dataset[ , receptor_transf_frecuente             := ifelse(mtransferencias_recibidas > 5,1,0)] ########## VERIFICAR PUNTO CORTE
    dataset[ , receptor_cheque_frecuente             := ifelse(ccheques_depositados > 5,1,0)] ########## VERIFICAR PUNTO CORTE

    #Autoservicio
    dataset[ , usa_autoservicio             := ifelse(cextraccion_autoservicio > 0,1,0)]

    #Cheques
    dataset[ , ccheques                := rowSums( cbind( ccheques_depositados,ccheques_emitidos ) , na.rm=TRUE ) ]
    dataset[ , mcheques                := rowSums( cbind( mcheques_depositados,mcheques_emitidos ) , na.rm=TRUE ) ]
    dataset[ , usa_cheques             := ifelse(ccheques > 0,1,0)]
    dataset[ , ccheques_rechazados                := rowSums( cbind(ccheques_depositados_rechazados,ccheques_emitidos_rechazados ) , na.rm=TRUE ) ]
    dataset[ , con_cheques_rechazados             := ifelse(ccheques_rechazados > 0,1,0)]

    #Cajeros
    dataset[ , catm_total_trx                := rowSums( cbind(catm_trx,catm_trx_other ) , na.rm=TRUE ) ]
    dataset[ , matm_total_trx                := rowSums( cbind(matm,matm_other ) , na.rm=TRUE ) ]
    
    dataset[ , ccualquier_cajero_trx                := rowSums( cbind(catm_total_trx,cextraccion_autoservicio ) , na.rm=TRUE ) ]
    dataset[ , mcualquier_cajero_trx                := rowSums( cbind(matm_total_trx,mextraccion_autoservicio ) , na.rm=TRUE ) ]
    dataset[ , usa_cualquier_cajero             := ifelse(ccualquier_cajero_trx > 0,1,0)]

    
    #Interacciones banco
     dataset[ , cinteracciones_banco_ult_mes    := rowSums( cbind(chomebanking_transacciones,ccajas_transacciones,ccallcenter_transacciones,cmobile_app_trx,ccualquier_cajero_trx ) , na.rm=TRUE ) ]
    
     dataset[ , prop_homeb_total                := ifelse(cinteracciones_banco_ult_mes==0,0,chomebanking_transacciones/cinteracciones_banco_ult_mes) ]
     dataset[ , prop_cajas_total                := ifelse(cinteracciones_banco_ult_mes==0,0,ccajas_transacciones/cinteracciones_banco_ult_mes) ]
     dataset[ , prop_callcenter_total           := ifelse(cinteracciones_banco_ult_mes==0,0,ccallcenter_transacciones/cinteracciones_banco_ult_mes) ]
     dataset[ , prop_app_total                  := ifelse(cinteracciones_banco_ult_mes==0,0,cmobile_app_trx/cinteracciones_banco_ult_mes) ]
     dataset[ , prop_cajero_total                  := ifelse(cinteracciones_banco_ult_mes==0,0,ccualquier_cajero_trx/cinteracciones_banco_ult_mes) ]

     dataset[ , prop_interac_mov                  := ifelse(ctrx_quarter==0,0,cinteracciones_banco_ult_mes/ctrx_quarter) ]
    dataset[ , sin_interac_ult_mes             := ifelse(cinteracciones_banco_ult_mes > 0,1,0)]
    dataset[ , sin_mov_ult_cuat             := ifelse(ctrx_quarter > 0,1,0)]

    #Payroll
   dataset[ , cpayroll_total    := rowSums( cbind(cpayroll_trx,cpayroll2_trx ) , na.rm=TRUE ) ]
   dataset[ , mpayroll_total    := rowSums( cbind(mpayroll,mpayroll2 ) , na.rm=TRUE ) ]
    dataset[ , acredita_sueldo  := ifelse(cpayroll_total > 0,1,0)]

    #Rentabilidad y productos
    dataset[ , mmargen    := rowSums( cbind(mactivos_margen,mpasivos_margen ) , na.rm=TRUE ) ]

    dataset[ , prop_rent_mes_anual                  := ifelse(mrentabilidad_annual==0,0,mrentabilidad/mrentabilidad_annual) ]
    dataset[ , prop_rent_mes_antiguedad                  := ifelse(cliente_antiguedad==0,0,mrentabilidad/cliente_antiguedad) ]
    dataset[ , prop_rent_mes_cprod                  := ifelse(cproductos==0,0,mrentabilidad/cproductos) ]
    dataset[ , prop_rent_anual_cprod                  := ifelse(cproductos==0,0,mrentabilidad_annual/cproductos) ]
    dataset[ , prop_comisiones_cprod                  := ifelse(cproductos==0,0,mcomisiones/cproductos) ]
    dataset[ , prop_comisiones_rent_mes                  := ifelse(mrentabilidad==0,0,mcomisiones/mrentabilidad) ]
    dataset[ , prop_margen_cprod                  := ifelse(cproductos==0,0,mmargen/cproductos) ]
    dataset[ , prop_margen_rent_mes                  := ifelse(mrentabilidad==0,0,mmargen/mrentabilidad) ]
    
    dataset[ , paga_comisiones  := ifelse(mcomisiones > 0,1,0)]
    dataset[ , da_margen  := ifelse(mmargen > 0,1,0)]
    dataset[ , es_rentable  := ifelse(mrentabilidad > 0,1,0)]

    #Debito y credito
    dataset[ , ctarjeta_credito    := rowSums( cbind(ctarjeta_visa,ctarjeta_master ) , na.rm=TRUE ) ]
    dataset[ , tiene_tarjeta_credito  := ifelse(ctarjeta_credito > 0,1,0)]
    dataset[ , ctarjeta_credito_transacciones    := rowSums( cbind(ctarjeta_visa_transacciones,ctarjeta_master_transacciones ) , na.rm=TRUE ) ]
    dataset[ , mtarjeta_credito_consumos    := rowSums( cbind(mtarjeta_visa_consumo,mtarjeta_master_consumo ) , na.rm=TRUE ) ]

    dataset[ , usa_tarjeta_credito  := ifelse(ctarjeta_credito_transacciones > 0,1,0)]
    dataset[ , usa_tarjeta_debito  := ifelse(ctarjeta_debito_transacciones > 0,1,0)]

    dataset[ , prop_tarj_debito_credito    := ifelse(ctarjeta_credito_transacciones==0,0,ctarjeta_debito_transacciones/ctarjeta_credito_transacciones) ]

    #Saldo
    dataset[ , prop_dolares_saldo   := ifelse(mcuentas_saldo==0,0,mcaja_ahorro_dolares/mcuentas_saldo) ]
    dataset[ , prop_plazos_fijos_saldo   := ifelse(mcuentas_saldo==0,0,mplazo_fijo/mcuentas_saldo) ]
    dataset[ , prop_inversion12_saldo   := ifelse(mcuentas_saldo==0,0,minversion12/mcuentas_saldo) ]
    dataset[ , prop_consumos_tarj_cred_saldo    := ifelse(mcuentas_saldo==0,0,mtarjeta_credito_consumos/mcuentas_saldo) ]
    dataset[ , prop_rent_mes_saldo   := ifelse(mcuentas_saldo==0,0,mrentabilidad/mcuentas_saldo) ]

    dataset[ , mcuentas_saldo_mas_invers    := rowSums( cbind(mcuentas_saldo,minversion_total ) , na.rm=TRUE ) ]
    dataset[ , mcuentas_saldo_mas_invers_mas_caja_seg    := ifelse(ccaja_seguridad==1,1000000,0) + mcuentas_saldo_mas_invers ]
    dataset[ , mgasto_mensual_estim    := rowSums( cbind(mdebitos_automaticos,mpago_servicios_total,mtransferencias_emitidas,mextraccion_autoservicio,mcheques_emitidos,ccajas_extracciones,mautoservicio,mtarjeta_credito_consumos ) , na.rm=TRUE ) ]
    dataset[ , mingreso_mensual_estim    := rowSums( cbind(mtransferencias_recibidas,mcheques_depositados,ccajas_depositos,mpayroll_total ) , na.rm=TRUE ) ]
    
    dataset[ , prop_gasto_saldo   := ifelse(mcuentas_saldo==0,0,mgasto_mensual_estim/mcuentas_saldo) ]
    dataset[ , prop_ingreso_saldo   := ifelse(mcuentas_saldo==0,0,mingreso_mensual_estim/mcuentas_saldo) ]

    #Visa y Master
    dataset[ , mv_status       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
    dataset[ , mv_delinquency       := pmax( Master_delinquency,  Visa_delinquency, na.rm = TRUE) ]
    dataset[ , mv_mfinanciacion_limite    := rowSums( cbind(Master_mfinanciacion_limite,Visa_mfinanciacion_limite ) , na.rm=TRUE ) ]
    dataset[ , mv_msaldototal    := rowSums( cbind(Master_msaldototal,Visa_msaldototal ) , na.rm=TRUE ) ]
    dataset[ ,  mv_msaldopesos   := rowSums( cbind(Master_msaldopesos,Visa_msaldopesos ) , na.rm=TRUE ) ]
    dataset[ ,  mv_msaldodolares   := rowSums( cbind(Master_msaldodolares, Visa_msaldodolares) , na.rm=TRUE ) ]
    dataset[ ,  mv_mconsumospesos   := rowSums( cbind(Master_mconsumospesos,Visa_mconsumospesos ) , na.rm=TRUE ) ]
    dataset[ ,  mv_mconsumosdolares   := rowSums( cbind(Master_mconsumosdolares,Visa_mconsumosdolares ) , na.rm=TRUE ) ]
    dataset[ ,  mv_mlimitecompra   := rowSums( cbind(Master_mlimitecompra, Visa_mlimitecompra) , na.rm=TRUE ) ]
    dataset[ ,  mv_madelantopesos   := rowSums( cbind(Master_madelantopesos,Visa_madelantopesos ) , na.rm=TRUE ) ]
    dataset[ ,  mv_madelantodolares   := rowSums( cbind(Master_madelantodolares,Visa_madelantodolares ) , na.rm=TRUE ) ]
    dataset[ ,  mv_mpagado   := rowSums( cbind(Master_mpagado,Visa_mpagado ) , na.rm=TRUE ) ]
    dataset[ ,  mv_mpagospesos   := rowSums( cbind(Master_mpagospesos,Visa_mpagospesos ) , na.rm=TRUE ) ]
    dataset[ ,   mv_mpagosdolares  := rowSums( cbind(Master_mpagosdolares,Visa_mpagosdolares ) , na.rm=TRUE ) ]
    dataset[ ,   mv_mconsumototal  := rowSums( cbind(Master_mconsumototal,Visa_mconsumototal ) , na.rm=TRUE ) ]
    dataset[ ,   mv_cconsumos  := rowSums( cbind(Master_cconsumos,Visa_cconsumos ) , na.rm=TRUE ) ]
    dataset[ ,   mv_cadelantosefectivo  := rowSums( cbind(Master_cadelantosefectivo,Visa_cadelantosefectivo ) , na.rm=TRUE ) ]
    dataset[ ,   mv_mpagominimo  := rowSums( cbind(Master_mpagominimo,Visa_mpagominimo ) , na.rm=TRUE ) ]

    dataset[ , prop_limite_compra_saldo   := ifelse(mcuentas_saldo==0,0,mv_mlimitecompra/mcuentas_saldo) ]
    dataset[ , prop_limite_compra_gastos   := ifelse(mgasto_mensual_estim==0,0,mv_mlimitecompra/mgasto_mensual_estim) ]
    dataset[ , prop_limite_compra_ingresos   := ifelse(mingreso_mensual_estim==0,0,mv_mlimitecompra/mingreso_mensual_estim) ]
    dataset[ , prop_limite_compra_saldo_estim   := ifelse(mcuentas_saldo_mas_invers_mas_caja_seg==0,0,mv_mlimitecompra/mcuentas_saldo_mas_invers_mas_caja_seg) ]

    #Productos de dificil desvinculacion
    dataset[ ,   cant_prod_dif_desvinc  := rowSums( cbind(cdebitos_automaticos,cseguro,cinversion12,cprestamos ) , na.rm=TRUE ) ]

    
    columnas_extendidas <-  copy( setdiff(  colnames(dataset), columnas_originales ) )

    return (dataset)
}
#------------------------------------------------------------------------------
dataset_enriq <- EnriquecerDataset( dataset_orig )


In [5]:
as.data.table(sapply(dataset_enriq[, list(mttarjeta_visa_debitos_automaticos)], sample, 10))

mttarjeta_visa_debitos_automaticos
<dbl>
2613.97
2985.28
10085.93
4852.26
36830.10
6014.69
10374.01
0.00
0.00


In [6]:
head(dataset_orig)

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,tarj_credito_mpagosdolares,tarj_credito_mconsumototal,tarj_credito_cconsumos,tarj_credito_cadelantosefectivo,tarj_credito_mpagominimo,prop_limite_compra_saldo,prop_limite_compra_gastos,prop_limite_compra_ingresos,prop_limite_compra_saldo_estim,cant_prod_dif_desvinc
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
31115668,202101,1,0,0,56,132,1897.12,24324.75,940.32,⋯,0.00,18435.59,1,0,1337.22,1.118721,1.804695,0.6512564,1.118721,4
31115678,202101,1,0,0,48,100,-384.72,33860.45,-515.53,⋯,0.00,0.00,0,0,0.00,0.000000,0.000000,0.0000000,0.000000,2
31117415,202101,1,0,0,60,53,-3622.66,-21310.32,4064.85,⋯,0.00,21182.80,6,0,20152.14,1.642494,1.780312,1.1214953,1.642494,2
31117711,202101,1,0,0,46,273,14542.05,125727.99,1037.77,⋯,14.27,758.21,3,0,1302.03,4.238328,14.654054,12.1098966,4.238328,9
31118145,202101,1,0,0,47,192,1868.27,37305.29,2861.62,⋯,0.00,6316.13,1,0,4680.27,14.582133,3.086385,16.7739531,14.582133,0
31118204,202101,1,0,0,68,258,2301.86,60781.98,5169.05,⋯,0.00,37036.92,17,0,4093.77,18.133433,4.933004,8.3994629,18.133433,23


In [7]:
  #grabo con nombre extendido
  fwrite( dataset_enriq,
          file=karchivo_salida,
          sep= "," )